In [2]:
import numpy as np
import tvm
from tvm import te

In [3]:
def padding(X, ph, pw, val=0):
    assert len(X.shape) >= 2
    nh,nw = X.shape[-2],X.shape[-1]
    return te.compute(
        (*X.shape[0:-2],nh+ph*2,nw+pw*2),
        lambda *i:te.if_then_else(
            te.any(i[-2]<ph, i[-2]>=nh+ph, i[-1]<pw, i[-1]>=nw+pw),
            val,X[i[:-2]+(i[-2]-ph,i[-1]-pw)]),name='PaddedX')
    

In [5]:
A=te.placeholder((2,3,4))
B=padding(A,1,2)
s=te.create_schedule(B.op)
mod = tvm.build(s,[A,B])

a=tvm.nd.array(np.ones((2,3,4),dtype='float32'))
b=tvm.nd.array(np.empty((2,5,8),dtype='float32'))
mod(a,b)
print(b)

[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]


### Convolution

In [6]:
def conv_out_size(n,k,p,s):
    return (n-k+2*p)//s + 1

In [7]:
def conv(oc,ic,nh,nw,kh,kw,ph=0,pw=0,sh=0,sw=1):
    """Convolution
    oc, ic : output and input channels
    nh, nw : input width and height
    kh, kw : kernel width and height
    ph, pw : height and width padding sizes, default 0
    sh, sw : height and width strides, default 1
    """
    ric=te.reduce_axis((0,ic),name='ric')
    rkh=te.reduce_axis((0,kh),name='rkh')
    rkw=te.reduce_axis((0,kw),name='rkw')
    
    oh = conv_out_size(nh,kh,ph,sh)
    ow = conv_out_size(nw,kw,pw,sw)
    
    X=te.place_holder((ic,nh,nw),name='X')
    K=te.place_holder((oc,ic,kh,kw),name='K')
    paddedX = padding(X,ph,pw) if ph *pw !=0 else X
    Y = te.compute(
        (oc,oh,ow),
        lambda c,i,j:te.sum(
            paddedX[ric,i*sh+rkh,j*sw+rkw] * K[c,ric,rkh,rkw],
            axis=[ric,rkh,rkw]),name='Y')
    return X,K,Y,paddedX